In [ ]:
# Cell 1: Import necessary libraries
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark import Session

In [ ]:
-- # Cell 2: No need for connection setup in Snowflake notebooks
-- # session is already available

-- # Cell 3: Create schema for harmonized data
USE ROLE CRYPTO_ROLE;
CREATE SCHEMA IF NOT EXISTS CRYPTO_DB.HARMONIZED_CRYPTO;


In [ ]:

CREATE OR REPLACE TABLE CRYPTO_DB.HARMONIZED_CRYPTO.CRYPTO_HARMONIZED (
    crypto_symbol VARCHAR(10),
    timestamp TIMESTAMP_NTZ,
    date_day DATE,
    open FLOAT,
    high FLOAT,
    low FLOAT,
    close FLOAT,
    volume FLOAT,
    adj_close FLOAT,
    price_change_24h FLOAT,
    price_change_percentage_24h FLOAT,
    volatility_7d FLOAT,
    normalized_price FLOAT,
    PRIMARY KEY (crypto_symbol, timestamp)
);

In [ ]:
# Cell 5: Transform and harmonize Yahoo Finance crypto data
def transform_yahoo_finance_data(session = None):
    if session is None:
        from snowflake.snowpark import Session
        session = Session.builder.getOrCreate()
    
    import snowflake.snowpark.functions as F
    import snowflake.snowpark.types as T
    
    # Create references to raw tables
    btc = session.table("CRYPTO_DB.RAW_CRYPTO.BTC")
    eth = session.table("CRYPTO_DB.RAW_CRYPTO.ETH")
    doge = session.table("CRYPTO_DB.RAW_CRYPTO.DOGE")
    
    # Function to standardize each cryptocurrency dataframe
    def standardize_crypto_df(df, symbol):
        return df.withColumn("crypto_symbol", F.lit(symbol)) \
                .withColumn("date_day", F.to_date(F.col('"date"'))) \
                .withColumn("price_change_24h", F.col('"close"') - F.col('"open"')) \
                .withColumn("price_change_percentage_24h", 
                            (F.col('"close"') - F.col('"open"')) / F.col('"open"') * 100)
    
    # Standardize each dataframe
    btc_std = standardize_crypto_df(btc, "BTC")
    eth_std = standardize_crypto_df(eth, "ETH")
    doge_std = standardize_crypto_df(doge, "DOGE")
    
    # Union all data into a single DataFrame
    all_crypto = btc_std.unionAll(eth_std).unionAll(doge_std)
    
    # Standardize column names and structure
    harmonized = all_crypto.select(
        F.col("crypto_symbol"),
        F.col('"date"').alias("timestamp"),
        F.col("date_day"),
        F.col('"open"').alias("open"),
        F.col('"high"').alias("high"),
        F.col('"low"').alias("low"),
        F.col('"close"').alias("close"),
        F.col('"volume"').alias("volume"),
        F.col('"adjclose"').alias("adj_close"),
        F.col("price_change_24h"),
        F.col("price_change_percentage_24h")
    )
    
    # Remove duplicates
    harmonized = harmonized.dropDuplicates(["crypto_symbol", "timestamp"])
    
    # Add the missing columns that match the table schema
    harmonized = harmonized.withColumn("volatility_7d", F.lit(None).cast(T.FloatType())) \
                          .withColumn("normalized_price", F.lit(None).cast(T.FloatType()))
    
    return harmonized

In [ ]:
-- # Cell 6: Create SQL UDF for normalizing currency exchange rates
-- %%sql
CREATE OR REPLACE FUNCTION CRYPTO_DB.HARMONIZED_CRYPTO.NORMALIZE_CURRENCY(price FLOAT, from_currency VARCHAR, to_currency VARCHAR)
RETURNS FLOAT
LANGUAGE SQL
AS
$$
    CASE 
        WHEN from_currency = to_currency THEN price
        WHEN from_currency = 'USD' AND to_currency = 'EUR' THEN price * 0.92
        WHEN from_currency = 'USD' AND to_currency = 'JPY' THEN price * 110.5
        ELSE price
    END
$$;

In [ ]:
CREATE OR REPLACE FUNCTION CRYPTO_DB.HARMONIZED_CRYPTO.CALCULATE_VOLATILITY(prices ARRAY)
RETURNS FLOAT
LANGUAGE PYTHON
RUNTIME_VERSION = '3.8'
HANDLER = 'calculate_volatility'
AS
$$
import math

def calculate_volatility(prices):
    if not prices or len(prices) < 2:
        return None
    
    # Calculate daily returns manually
    daily_returns = []
    for i in range(1, len(prices)):
        if prices[i-1] != 0:  # Avoid division by zero
            daily_return = (prices[i] - prices[i-1]) / prices[i-1]
            daily_returns.append(daily_return)
    
    if not daily_returns:
        return None
    
    # Calculate mean
    mean = sum(daily_returns) / len(daily_returns)
    
    # Calculate variance
    variance = sum((x - mean) ** 2 for x in daily_returns) / len(daily_returns)
    
    # Calculate standard deviation
    std_dev = math.sqrt(variance)
    
    # Annualize (assuming 252 trading days)
    volatility = std_dev * math.sqrt(252)
    
    return float(volatility)
$$;

In [ ]:
CREATE OR REPLACE PROCEDURE CRYPTO_DB.HARMONIZED_CRYPTO.UPDATE_VOLATILITY_METRICS()
RETURNS STRING
LANGUAGE SQL
AS
$$
BEGIN
    -- Create a temporary table with calculated volatility
    CREATE OR REPLACE TEMPORARY TABLE temp_volatility AS
    SELECT 
        t1.crypto_symbol,
        t1.timestamp,
        CRYPTO_DB.HARMONIZED_CRYPTO.CALCULATE_VOLATILITY(
            ARRAY_AGG(t2.close) WITHIN GROUP (ORDER BY t2.timestamp)
        ) AS calc_volatility
    FROM 
        CRYPTO_DB.HARMONIZED_CRYPTO.CRYPTO_HARMONIZED t1
    JOIN 
        CRYPTO_DB.HARMONIZED_CRYPTO.CRYPTO_HARMONIZED t2
    ON 
        t1.crypto_symbol = t2.crypto_symbol
        AND t2.timestamp BETWEEN DATEADD(day, -7, t1.timestamp) AND t1.timestamp
    GROUP BY 
        t1.crypto_symbol, t1.timestamp
    HAVING 
        COUNT(*) >= 2;
    
    -- Update volatility using the temporary table
    UPDATE CRYPTO_DB.HARMONIZED_CRYPTO.CRYPTO_HARMONIZED h
    SET volatility_7d = t.calc_volatility
    FROM temp_volatility t
    WHERE h.crypto_symbol = t.crypto_symbol
    AND h.timestamp = t.timestamp;
    
    -- Set normalized_price to equal the close price (since all data is already in USD)
    UPDATE CRYPTO_DB.HARMONIZED_CRYPTO.CRYPTO_HARMONIZED
    SET normalized_price = close;
    
    -- Drop the temporary table
    DROP TABLE IF EXISTS temp_volatility;
    
    RETURN 'Volatility metrics and normalized prices updated successfully';
END;
$$;

In [ ]:
# Cell 9: Example of executing the harmonization process
# Note: This would need real data in your raw tables to work
try:
    # Get a Snowpark session
    from snowflake.snowpark import Session
    session = Session.builder.getOrCreate()
    
    # Transform data
    print("Beginning data transformation...")
    harmonized_df = transform_yahoo_finance_data()
    
    # Save to harmonized table
    print("Saving to harmonized table...")
    harmonized_df.write.mode("overwrite").saveAsTable("CRYPTO_DB.HARMONIZED_CRYPTO.CRYPTO_HARMONIZED")
    
    # Update metrics
    print("Updating volatility metrics...")
    session.sql("CALL CRYPTO_DB.HARMONIZED_CRYPTO.UPDATE_VOLATILITY_METRICS()").collect()
    
    print("Data harmonization complete!")
except Exception as e:
    print(f"Error during harmonization: {str(e)}")
    print("Please ensure your raw tables exist and contain the expected columns.")

In [ ]:
CREATE OR REPLACE PROCEDURE CRYPTO_DB.HARMONIZED_CRYPTO.HARMONIZE_CRYPTO_DATA_SP()
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = '3.8'
HANDLER = 'harmonize_crypto_data'
PACKAGES = ('snowflake-snowpark-python')
AS
$$
def harmonize_crypto_data(session):
    import snowflake.snowpark.functions as F
    import snowflake.snowpark.types as T
    
    try:
        # Create references to raw tables
        btc = session.table("CRYPTO_DB.RAW_CRYPTO.BTC")
        eth = session.table("CRYPTO_DB.RAW_CRYPTO.ETH")
        doge = session.table("CRYPTO_DB.RAW_CRYPTO.DOGE")
        
        # Function to standardize each cryptocurrency dataframe
        def standardize_crypto_df(df, symbol):
            # Use quoted column names
            return df.withColumn("crypto_symbol", F.lit(symbol)) \
                    .withColumn("date_day", F.to_date(F.col('"date"'))) \
                    .withColumn("price_change_24h", F.col('"close"') - F.col('"open"')) \
                    .withColumn("price_change_percentage_24h", 
                                (F.col('"close"') - F.col('"open"')) / F.col('"open"') * 100)
        
        # Standardize each dataframe
        btc_std = standardize_crypto_df(btc, "BTC")
        eth_std = standardize_crypto_df(eth, "ETH")
        doge_std = standardize_crypto_df(doge, "DOGE")
        
        # Union all data into a single DataFrame
        all_crypto = btc_std.unionAll(eth_std).unionAll(doge_std)
        
        # Standardize column names and structure
        harmonized = all_crypto.select(
            F.col("crypto_symbol"),
            F.col('"date"').alias("timestamp"),
            F.col("date_day"),
            F.col('"open"'),
            F.col('"high"'),
            F.col('"low"'),
            F.col('"close"'),
            F.col('"volume"'),
            F.lit(None).cast(T.FloatType()).alias("adj_close"),  # Modified since adjclose doesn't exist
            F.col("price_change_24h"),
            F.col("price_change_percentage_24h")
        )
        
        # Remove duplicates
        harmonized = harmonized.dropDuplicates(["crypto_symbol", "timestamp"])
        
        # Add the missing columns that match the table schema
        harmonized = harmonized.withColumn("volatility_7d", F.lit(None).cast(T.FloatType())) \
                              .withColumn("normalized_price", F.lit(None).cast(T.FloatType()))
        
        # Write to harmonized table 
        # Use merge mode to handle incremental updates
        harmonized.write.mode("append").saveAsTable("CRYPTO_DB.HARMONIZED_CRYPTO.CRYPTO_HARMONIZED")
        
        # Update the volatility metrics
        session.sql("CALL CRYPTO_DB.HARMONIZED_CRYPTO.UPDATE_VOLATILITY_METRICS()").collect()
        
        return "Data harmonization completed successfully"
    except Exception as e:
        return f"Error during harmonization: {str(e)}"
$$;
-- Grant necessary permissions
GRANT USAGE ON PROCEDURE CRYPTO_DB.HARMONIZED_CRYPTO.HARMONIZE_CRYPTO_DATA_SP() TO ROLE CRYPTO_ROLE;

In [ ]:
-- Refrence---
SELECT *
FROM CRYPTO_DB.HARMONIZED_CRYPTO.CRYPTO_HARMONIZED
WHERE DATE_DAY = '2024-02-05'
ORDER BY CRYPTO_SYMBOL;